In [ ]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import os

from functools import partial

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import add
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.3.0


In [ ]:
!pip install face-alignment

In [ ]:
import face_alignment

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, face_detector='sfd')


In [ ]:
fa.face_detector.reference_scale

195

In [ ]:
detected_faces = fa.face_detector.detect_from_image(input[..., ::-1].copy())

In [ ]:
detected_faces[0]

array([372.92505, 136.7204 , 746.49475, 660.4481 ,   1.     ],
      dtype=float32)

In [ ]:
from skimage import io
input = io.imread('/content/Capture.PNG')
input = input[:,:,:3]
res = fa.get_landmarks(input)

In [ ]:
len(res[0])

68

In [ ]:
!pip install keras-facenet


     |████████████████████████████████| 2.3MB 9.3MB/s 
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-cp36-none-any.whl size=10387 sha256=26483e558bfeb974c6d5a2e1c0b6d2ce93e5322adb644e7a840fc28968e64361
  Stored in directory: /root/.cache/pip/wheels/f6/53/9a/36c4b52fd22faf4f710d5047d874655b85a1b2cf77accfb9bd
Successfully built keras-facenet


In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras import regularizers
import tensorflow as tf
initializer = tf.keras.initializers.HeUniform()
inputs = Input(shape=(160, 160, 3))
out = embedder.model(inputs)
X = Flatten()(out)
# X = Dense(256,  kernel_initializer = glorot_uniform(seed=0),kernel_regularizer=regularizers.l2(1e-4))(X)
# X = BatchNormalization()(X)
# X = Activation('relu')(X)
X = Dropout(0.2)(X)
out = Dense(7, activation='softmax', name='fc7', kernel_initializer=initializer )(X)
model = Model(inputs, out)

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
inception_resnet_v1 (Functio (None, 512)               23497424  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
fc7 (Dense)                  (None, 7)                 3591      
Total params: 23,501,015
Trainable params: 23,471,415
Non-trainable params: 29,600
_________________________________________________________________


In [ ]:
model_path = "/content/drive/MyDrive/btl_dl/final_dl/facenet_0.69.hdf5"

In [ ]:
for layer in model.layers[1].layers[:-25] :
  layer.trainable = False

In [ ]:
model.load_weights(model_path)

In [ ]:
model = None
from tensorflow.keras.applications import EfficientNetB2
eff = EfficientNetB2(include_top = False, input_shape=(96, 96, 3), pooling='avg')

31793152/31790344 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras import regularizers

inputs = Input(shape=(96, 96, 3))
out = eff(inputs)
X = Flatten()(out)
X = Dropout(0.2)(X)
# X = Dense(32,  kernel_initializer = glorot_uniform(seed=0),kernel_regularizer=regularizers.l2(1e-4))(X)
# X = BatchNormalization()(X)
# X = Activation('relu')(X)
# X = Dropout(0.2)(X)
out = Dense(7, activation='softmax', name='fc7', kernel_initializer = glorot_uniform(seed=0))(X)
model = Model(inputs, out)

In [ ]:
model_path = "/content/drive/MyDrive/btl_dl/final_dl/eff.hdf5"
model.load_weights(model_path)

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
import cv2
import numpy as np
mtcnn = FaceNet()
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
index2label = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('/content/WIN_20201212_10_59_40_Pro.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
cap.set(cv2.CAP_PROP_FPS, 30)
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
from google.colab.patches import cv2_imshow

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    # cv2.imshow('Frame',frame)
    image = cv2.flip(frame,1)
    # detections, imgs = mtcnn.crop(image, threshold=0.5)
    detected_faces = fa.face_detector.detect_from_image(image[..., ::-1].copy())
    # if len(imgs) > 0:
      # detections = sorted(enumerate(detections), key = lambda x : x[1]['confidence'], reverse = True)
    if len(detected_faces) >0 :
      detected_faces = sorted(enumerate(detected_faces), key = lambda x : x[1][-1], reverse = True)[0][1]
      bb = detected_faces[:-1]
      # bb = detections[0][1]['box']
      # idx = detections[0][0]
      # img = imgs[idx]
      w = bb[2]-bb[0]
      h = bb[3]-bb[1]
      height,width, _ = image.shape
      img = image[max(0,int(bb[1]-h*0.15)): int(min(bb[3]+h*0.15,height)), int(max(0,bb[0]-w*0.15)): int(min(bb[2]+w*0.15,width)), :]
      # img = cv2.resize(img,(160,160))
      img = cv2.resize(img,(96,96))
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
      # nor_img = (gray - 127.5) / 127.5
      nor_img = gray
      nor_img = np.stack((nor_img,nor_img,nor_img)).transpose(1,2,0)
      nor_img = preprocess_input(nor_img)
      res = model.predict(np.array([nor_img]))
      idx = np.argmax(res[0])
      emo = index2label[idx]
      x1, y1, x2, y2 = bb
      # x1, y1, w, h = bb
      # x2, y2 = x1+w, y1+h
      cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0),2)
      front_scale=(w + h)/130
      thickness =int(front_scale / 4)+1
      if thickness < 1 :
          thickness=1
      if front_scale <1:
          front_scale=1
      cv2.putText(image, emo, (x1,y2), cv2.FONT_HERSHEY_PLAIN,front_scale,(255,255,255),thickness,cv2.LINE_AA)
      # cv2_imshow(image)
      cv2.imwrite("now.jpg",image)
    out.write(image)

    # if len(detections) != 0 :
    #   detections = sorted(test, key = lambda x : x['confidence'], reverse = True)
    #   res = detections[0]

    # print(detections)
    # break
    # Press Q on keyboard to  exit
    # if cv2.waitKey(25) & 0xFF == ord('q'):
    #   break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()
